## GOAL
Load helper functions for plots and simulation.


### What you should notice
We reuse the same helper functions for all visuals.


In [ ]:
# Helper functions for visualization and simulation.
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import Statevector

def show_circuit(circuit):
    """Show a circuit diagram (mpl if available, else text)."""
    try:
        return circuit.draw('mpl')
    except Exception:
        print(circuit)

def show_bloch(circuit):
    """Show Bloch sphere for a 1-qubit circuit state."""
    state = Statevector.from_instruction(circuit)
    return plot_bloch_multivector(state)

def run_and_plot_counts(circuit, shots=1024):
    """Run a circuit on a simple simulator and plot counts."""
    try:
        from qiskit.primitives import StatevectorSampler
        sampler = StatevectorSampler()
    except Exception:
        from qiskit.primitives import Sampler
        sampler = Sampler()
    result = sampler.run([circuit], shots=shots).result()
    quasi = result.quasi_dists[0]
    counts = {k: int(v * shots) for k, v in quasi.items()}
    print("Counts:", counts)
    return plot_histogram(counts)

from qiskit import QuantumCircuit


## EXERCISE
No exercise here. Continue to the next cell.


In [ ]:
# Student solution cell (optional).


## GOAL
Create a Bell state and observe simulator counts.


### What you should notice
Only 00 and 11 appear with high probability.


In [ ]:
# Bell state on simulator.
bell = QuantumCircuit(2, 2)
bell.h(0)
bell.cx(0, 1)
bell.measure([0, 1], [0, 1])
show_circuit(bell)
run_and_plot_counts(bell)


## EXERCISE
Compute the correlation rate: (00 + 11) / total.


In [ ]:
# Student solution cell (optional).


## GOAL
Superdense coding: send 2 classical bits using 1 qubit + entanglement.


### What you should notice
Each message decodes to a unique bitstring.

Encoding table:
- 00 -> I
- 01 -> X
- 10 -> Z
- 11 -> XZ


In [ ]:
# Superdense coding demo.
def superdense(msg):
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    if msg == "01":
        qc.x(0)
    elif msg == "10":
        qc.z(0)
    elif msg == "11":
        qc.x(0)
        qc.z(0)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure([0, 1], [0, 1])
    return qc

for msg in ["00", "01", "10", "11"]:
    qc = superdense(msg)
    show_circuit(qc)
    run_and_plot_counts(qc)


## EXERCISE
Test all four messages and verify the decoded output.


In [ ]:
# Student solution cell (optional).


## GOAL
Optional: Run on a real IBM QPU (end of workshop).


### What you should notice
Real hardware results are noisy and queues are normal. Simulator output is the baseline.

You need:
- An IBM Quantum account
- QISKIT_IBM_TOKEN set in your environment
- Optional install: pip install qiskit-ibm-runtime


In [ ]:
# Optional QPU run (only here do we import IBM Runtime).
import os
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

token = os.getenv("QISKIT_IBM_TOKEN")
if not token:
    raise SystemExit("Token missing. Set QISKIT_IBM_TOKEN and restart the kernel.")

service = QiskitRuntimeService(channel="ibm_quantum", token=token)
backend = service.least_busy(simulator=False, operational=True)
print("Using backend:", backend.name)

# Use the Bell circuit from above as a baseline.
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
bell_t = pm.run(bell)

sampler = SamplerV2(backend=backend)
job = sampler.run([bell_t], shots=1024)
print("Job ID:", job.job_id())
print("Queue delays are normal. Compare QPU output to simulator baseline.")
result = job.result()
quasi = result.quasi_dists[0]
counts = {k: int(v * 1024) for k, v in quasi.items()}
plot_histogram(counts)


## EXERCISE
Optional: compare QPU counts to simulator counts and describe differences.


In [ ]:
# Student solution cell (optional).
